# Redes neuronales: CNN


## 1.- Descripción

Las Redes Neuronales Convolucionales (CNN, por sus siglas en inglés) son un tipo especial de red neuronal que se ha demostrado ser extremadamente eficaz para el procesamiento de datos estructurados en forma de grillas, como imágenes, series temporales y datos de audio. Este modelo se inspira en el sistema visual de los seres humanos, lo que le permite identificar patrones jerárquicos de manera eficiente, desde características simples (como bordes y texturas) hasta patrones más complejos (como formas o objetos completos).

En este proyecto, el objetivo es aplicar una red neuronal convolucional para un problema de clasificación binaria. Utilizaremos las capacidades de las CNN para extraer características relevantes del conjunto de datos y realizar una clasificación precisa, optimizando tanto las métricas de precisión como de recall, especialmente cuando se trata de un conjunto de datos desbalanceado. Se busca que el modelo no solo sea preciso, sino que también sea capaz de manejar los datos desbalanceados de manera efectiva, garantizando una clasificación más robusta en ambas clases.

Este enfoque de modelado con CNN puede ser útil para diversas aplicaciones, no solo en visión por computadora, sino también en dominios como la clasificación de textos, predicción de series temporales, y en cualquier escenario donde las relaciones espaciales y jerárquicas sean importantes.

## 2.- Implementación


In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.optimizers import Adam

In [87]:
# Cargar los datos
df = pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\df_train.csv")
df_test= pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\raw\test_nolabel.csv")

In [89]:
# Balancear el dataset con undersampling
df_majority = df[df["Accept"] == 1]
df_minority = df[df["Accept"] == 0]
df_majority_downsampled = resample(df_majority, 
                                   replace=False,
                                   n_samples=len(df_minority),
                                   random_state=42)

In [91]:
df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [93]:
# Separar X e y
X = df_balanced.drop('Accept', axis=1)
y = df_balanced['Accept']

In [95]:
df_test_features = df_test.copy()

In [97]:
train_columns = X.columns

In [99]:
# Codificar texto si hay columnas categóricas
le = LabelEncoder()
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    X[col] = le.fit_transform(X[col].astype(str))


In [101]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns

categorical_columns_test = [col for col in categorical_columns_test if col != 'id']
for col in categorical_columns_test:
    if col in categorical_columns:  # Solo transformar las columnas que están en df_train
        # Reemplaza las categorías desconocidas con 'Unknown' si es necesario
        if 'Unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Unknown')
        
        # Reemplaza las categorías desconocidas con 'Unknown' y aplica la transformación
        df_test[col] = df_test[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
        df_test[col] = le.transform(df_test[col])

In [103]:
categorical_columns_test = df_test_features.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns_test:
    df_test_features[col] = le.fit_transform(df_test_features[col].astype(str))


In [105]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
df_test_imputed = imputer.fit_transform(df_test_features)

In [107]:
# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
df_test_scaled = scaler.fit_transform(df_test_imputed)

In [119]:
# Dividir en train/test

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



In [127]:
# Reshape para CNN (necesita 3D input: samples, time steps, features)
# Luego expandimos la dimensión solo para los conjuntos de entrenamiento y test
X_train = np.squeeze(X_train, axis=-1)  # Expande las dimensiones a (n_samples, n_features, 1)


ValueError: cannot select an axis to squeeze out which has size not equal to one

In [121]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (6129, 20)
Dimensiones de df_test_scaled: (3284, 20)


In [67]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de X_train: (6129, 20, 1, 1)
Dimensiones de df_test_scaled: (3284, 20)


## 3.- Evaluación y optimización


In [243]:
model = Sequential([
    Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train.shape[1], 1)),  # Capa convolucional
    MaxPooling1D(pool_size=2),  # Capa de MaxPooling
    Dropout(0.3),  # Dropout para evitar sobreajuste
    Flatten(),  # Aplanar las características
    Dense(128, activation='relu'),  # Capa densa con activación ReLU
    Dropout(0.3),  # Dropout para evitar sobreajuste
    Dense(1, activation='sigmoid')  # Capa de salida con activación sigmoide para clasificación binaria
])

# Compilar el modelo con la función de pérdida binary_crossentropy
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [245]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 16, 64)            384       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 8, 64)            0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 8, 64)             0         
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                      

In [247]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/30
154/154 [==============================] - 1s 4ms/step - loss: 0.6350 - accuracy: 0.6433 - val_loss: 0.6294 - val_accuracy: 0.6370
Epoch 2/30
154/154 [==============================] - 0s 3ms/step - loss: 0.6172 - accuracy: 0.6635 - val_loss: 0.6131 - val_accuracy: 0.6599
Epoch 3/30
154/154 [==============================] - 1s 4ms/step - loss: 0.6064 - accuracy: 0.6741 - val_loss: 0.5980 - val_accuracy: 0.6762
Epoch 4/30
154/154 [==============================] - 0s 3ms/step - loss: 0.6012 - accuracy: 0.6757 - val_loss: 0.5935 - val_accuracy: 0.6794
Epoch 5/30
154/154 [==============================] - 0s 3ms/step - loss: 0.5948 - accuracy: 0.6808 - val_loss: 0.5968 - val_accuracy: 0.6688
Epoch 6/30
154/154 [==============================] - 0s 3ms/step - loss: 0.5942 - accuracy: 0.6824 - val_loss: 0.5925 - val_accuracy: 0.6737
Epoch 7/30
154/154 [==============================] - 0s 3ms/step - loss: 0.5868 - accuracy: 0.6873 - val_loss: 0.5864 - val_accuracy: 0.6819
Epoch 

In [259]:
# Predecir probabilidades
y_pred= model.predict(X_test)


48/48 [==============================] - 0s 2ms/step


In [283]:
y_pred_prob = model.predict(X_test)

y_pred = (y_pred_prob > 0.4).astype(int)

48/48 [==============================] - 0s 2ms/step


In [285]:
# Comprobar las dimensiones de X_train y df_test_scaled
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)


Dimensiones de X_train: (6129, 20)
Dimensiones de df_test_scaled: (3284, 20)


In [287]:
# Hacer predicciones
y_test_pred = model.predict(df_test_scaled)
y_test_bin = (y_test_pred > 0.5).astype(int)


103/103 [==============================] - 0s 1ms/step


In [289]:
print("Dimensiones de df_test_scaled:", df_test_scaled.shape)

Dimensiones de df_test_scaled: (3284, 20)


In [291]:
# Evaluar
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[414 329]
 [159 631]]
              precision    recall  f1-score   support

           0       0.72      0.56      0.63       743
           1       0.66      0.80      0.72       790

    accuracy                           0.68      1533
   macro avg       0.69      0.68      0.68      1533
weighted avg       0.69      0.68      0.68      1533



In [293]:
# Crear DataFrame con las predicciones (id y Accept)
df_result = df_test[['id']].copy()  # Agregar 'id' del conjunto de test
df_result['ACCEPT'] = y_test_bin
# Verifica las primeras filas del DataFrame
print(df_result.head())

            id  ACCEPT
0  6b7ce9ec161       0
1  96123015731       1
2  b2c5181ac5b       0
3  e6cb54a9e6a       1
4  8eddf83466e       1


In [298]:
# Guardar el resultado en un archivo CSV
df_result.to_csv("SUBMMIT_CNN.csv", index=False)

print("¡Predicciones completas y archivo guardado como 'resultado_test.csv'!")


¡Predicciones completas y archivo guardado como 'resultado_test.csv'!


In [300]:
df_result.shape

(3284, 2)

In [302]:
# Verifica las dimensiones de df_test_scaled y y_test_pred
print(f"Dimensiones de df_test_scaled: {df_test_scaled.shape}")
print(f"Dimensiones de y_test_pred: {y_test_pred.shape}")


Dimensiones de df_test_scaled: (3284, 20)
Dimensiones de y_test_pred: (3284, 1)


## 4.- Conclusión

El modelo muestra un rendimiento razonable en la clasificación de préstamos bancarios, con una precisión y recall aceptables para ambas clases (rechazados y aprobados). El modelo tiene una mayor capacidad para identificar correctamente los casos de rechazo (con un buen recall de la clase 0), pero presenta un ligero desequilibrio en la identificación de préstamos aprobados, con un recall algo más bajo para la clase 1.

Aunque la exactitud global es del 68%, lo que indica que el modelo está clasificando correctamente una porción considerable de los casos, el desempeño podría mejorar, especialmente en la identificación de la clase minoritaria (préstamos aprobados). Los resultados sugieren que, aunque el modelo es funcional, se podrían realizar ajustes para mejorar el balance y la precisión, como ajustar el umbral de decisión o explorar otras técnicas de modelado.